data_preprocessing.ipynb
by: 
- Sheil Kumar (sk17@illinois.edu)
- Anirudh Eswara (aeswara2@illinois.edu)
- Lloyd Fernandes (lloydf2@illinois.edu)

Project director: Richard Sowers
r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/
Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license


# ETL on pNEUMA dataset
This notebook serves to show how the features in `data.py` and `feature_eng.py` is used to clean and augment the data downloaded from 



In [1]:
from data import *
from feature_eng import *

## 1.0 Data.py

- Functions:
  - `csv_to_df`
  - `pickle_df`

In [2]:
#The original dataset
pd.read_csv("block_1_sample.csv",nrows = 5)

,track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time
0,1; Taxi; 320.26; 35.366264; 37.977482; 23.7354...
1,2; Bus; 250.64; 24.653107; 37.978025; 23.73501...
2,3; Car; 257.14; 49.240297; 37.977918; 23.73499...
3,4; Motorcycle; 264.19; 45.725883; 37.977864; 2...
4,5; Car; 272.25; 33.336716; 37.977843; 23.73509...


In [3]:
#csv_to_df converts the above displayed csv to a multi Index dataframe
df = csv_to_df('block_1_sample.csv') #simply need to feed in name of .csv file containing dat a
pickle_df(df,pickle_name='block_1_sample_df.pkl') #simply need to feed in dataframe and desired name for .pkl file 
df.head()

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                      
1  0.00  37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04  37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08  37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12  37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16  37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   

         avg_speed  
id time             
1  0.00  35.366264  
   0.04  35.366264  
   0.08  35.366264  
   0.12  35.366264  
   0.16  35.366264

## 2.0 feature_eng.py

- Functions:
    - `bearing`
    - `nearest_graph_data`
    - `direction`
    - `vehicle_density` 
    - `cross_track`
    - `edge_average_speed`
    - `split_trajectories`

In [4]:
#Add bearings
df = bearing(df) #using bearing will then return a df with bearing as a column 
df.head()

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                      
1  0.00  37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04  37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08  37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12  37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16  37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   

         avg_speed   bearing  
id time                       
1  0.00  35.366264 -0.170905  
   0.04  35.366264 -0.114554  
   0.08  35.366264 -0.114553  
   0.12  35.366264 -0.170898  
   0.16  35.366264 -0.170896

In [5]:
#add nearest nodes and edges
lat = df.iloc[1,0]
lon = df.iloc[1,1]
graph = ox.graph_from_point((lat,lon), network_type='drive', dist=700)
#graph = ox.graph_from_address('Athens, Municipality of Athens, Regional Unit of Central Athens, Attica, 10667, Greece', network_type='drive')  #need to provide graph from osmnx as well 
df = nearest_graph_data(df, graph,mode='balltree') #nearest_graph_data will then return multiple features regarding the node and edge per id 
df.head()

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                      
1  0.00  37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04  37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08  37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12  37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16  37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   

         avg_speed   bearing  nearest_edge_start_node  nearest_edge_end_node  \
id time                                                                        
1  0.00  35.366264 -0.170905                250691723             3339821648   
   0.04  35.366264 -0.114554                250691723             3339821648   
   0.08  35.366264 -0.114553                250691723             3339821648   
   0.12  35.366264 -0.170898                250691723             3339821648   
   0.16  35.366264 -0.170896                250691723             3339821648   

         edge_progress  
id time                 
1  0.00       0.511504  
   0.04       0.508871  
   0.08       0.506555  
   0.12       0.504240  
   0.16       0.501607

In [6]:
# add vehicle direction wrt edge
df = direction(df) # running direction on dataframe will then return a df with the direction as a column 
df.head()

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                      
1  0.00  37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04  37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08  37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12  37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16  37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   

         avg_speed   bearing  nearest_edge_start_node  nearest_edge_end_node  \
id time                                                                        
1  0.00  35.366264 -0.170905                250691723             3339821648   
   0.04  35.366264 -0.114554                250691723             3339821648   
   0.08  35.366264 -0.114553                250691723             3339821648   
   0.12  35.366264 -0.170898                250691723             3339821648   
   0.16  35.366264 -0.170896                250691723             3339821648   

         edge_progress  dir  
id time                      
1  0.00       0.511504    0  
   0.04       0.508871    0  
   0.08       0.506555    0  
   0.12       0.504240    0  
   0.16       0.501607    0

In [7]:
#Add Cross track distance
df = cross_track(df,graph)
df.head()

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                      
1  0.00  37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04  37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08  37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12  37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16  37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   

         avg_speed   bearing  nearest_edge_start_node  nearest_edge_end_node  \
id time                                                                        
1  0.00  35.366264 -0.170905                250691723             3339821648   
   0.04  35.366264 -0.114554                250691723             3339821648   
   0.08  35.366264 -0.114553                250691723             3339821648   
   0.12  35.366264 -0.170898                250691723             3339821648   
   0.16  35.366264 -0.170896                250691723             3339821648   

         edge_progress  dir  xtrack_dist  
id time                                   
1  0.00       0.511504    0    -2.097232  
   0.04       0.508871    0    -2.066049  
   0.08       0.506555    0    -2.107500  
   0.12       0.504240    0    -2.148950  
   0.16       0.501607    0    -2.117767

In [8]:
#vehicle_density
vehicle_density_df = vehicle_density(df) #running vehicle_density then returns the number of vehicles per start_node and end_node pairs, still needs to be reformatted and added back to main df
vehicle_density_df.head()

id
                                                                                     nunique
nearest_edge_start_node nearest_edge_end_node dir edge_progress_intervals time_stamp        
42240105                250691272             0   0.0                     2.28             1
                                                                          2.32             1
                                                                          2.36             1
                                                                          2.40             1
                                                                          59.28            1

In [9]:
#integrate vehicle_density_df with df
col_list = list(df.columns)
df= df.reset_index().merge(vehicle_density_df,how = 'left', left_on=['time','nearest_edge_start_node','nearest_edge_end_node','dir','edge_progress_intervals'], 
         right_on =['time_stamp','nearest_edge_start_node','nearest_edge_end_node','dir','edge_progress_intervals'] )[['id','time']+ col_list + [('id', 'nunique')]]
df.set_index(['id','time'], inplace=True)
df.rename(columns = {('id', 'nunique') : 'vehicle_density'},inplace=True)
df.head()
#df["vehicle_speed"] = df_vehicle_speed[('id', 'nunique')]

C:\Users\lloyd\anaconda3\envs\ox\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\lloyd\anaconda3\envs\ox\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                      
1  0.00  37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04  37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08  37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12  37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16  37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   

         avg_speed   bearing  nearest_edge_start_node  nearest_edge_end_node  \
id time                                                                        
1  0.00  35.366264 -0.170905                250691723             3339821648   
   0.04  35.366264 -0.114554                250691723             3339821648   
   0.08  35.366264 -0.114553                250691723             3339821648   
   0.12  35.366264 -0.170898                250691723             3339821648   
   0.16  35.366264 -0.170896                250691723             3339821648   

         edge_progress  dir  xtrack_dist  time_stamp  edge_progress_intervals  \
id time                                                                         
1  0.00       0.511504    0    -2.097232        0.00                      0.5   
   0.04       0.508871    0    -2.066049        0.04                      0.5   
   0.08       0.506555    0    -2.107500        0.08                      0.5   
   0.12       0.504240    0    -2.148950        0.12                      0.5   
   0.16       0.501607    0    -2.117767        0.16                      0.5   

         vehicle_density  
id time                   
1  0.00                5  
   0.04                5  
   0.08                5  
   0.12                5  
   0.16                5

In [10]:
#average surrounding speed
average_surr_speed = edge_average_speed(df)
average_surr_speed.head()

nearest_edge_start_node  nearest_edge_end_node  edge_progress_intervals  dir  time_stamp
42240105                 250691272              0.0                      0    2.28          27.6670
                                                                              2.32          27.6433
                                                                              2.36          27.6202
                                                                              2.40          27.5970
                                                                              59.28         27.1494
Name: speed, dtype: float64

In [11]:
#integrate average surrounding speed
col_list = list(df.columns)
df= df.reset_index().merge(average_surr_speed,how = 'left', left_on=['time','nearest_edge_start_node','nearest_edge_end_node','dir','edge_progress_intervals'], 
         right_on =['time_stamp','nearest_edge_start_node','nearest_edge_end_node','dir','edge_progress_intervals'],suffixes = ['','_y'] )#[['id','time']+ col_list + ['speed_y']]
df.set_index(['id','time'], inplace=True)
df.rename(columns = {('speed_y') : 'avg_surr_speed'},inplace=True)
df.head()

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                      
1  0.00  37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04  37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08  37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12  37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16  37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   

         avg_speed   bearing  nearest_edge_start_node  nearest_edge_end_node  \
id time                                                                        
1  0.00  35.366264 -0.170905                250691723             3339821648   
   0.04  35.366264 -0.114554                250691723             3339821648   
   0.08  35.366264 -0.114553                250691723             3339821648   
   0.12  35.366264 -0.170898                250691723             3339821648   
   0.16  35.366264 -0.170896                250691723             3339821648   

         edge_progress  dir  xtrack_dist  time_stamp  edge_progress_intervals  \
id time                                                                         
1  0.00       0.511504    0    -2.097232        0.00                      0.5   
   0.04       0.508871    0    -2.066049        0.04                      0.5   
   0.08       0.506555    0    -2.107500        0.08                      0.5   
   0.12       0.504240    0    -2.148950        0.12                      0.5   
   0.16       0.501607    0    -2.117767        0.16                      0.5   

         vehicle_density  avg_surr_speed  
id time                                   
1  0.00                5        38.93726  
   0.04                5        39.07452  
   0.08                5        39.22120  
   0.12                5        39.37862  
   0.16                5        39.54336

In [12]:
#split trajectories to timestamp with 1500 entries
df = split_trajectories(df, 2000)

In [13]:
#get the relevant features and create a unique edge_id
df_list = df[['lat','lon','speed','lon_acc','lat_acc','type','traveled_d','avg_speed','bearing','edge_progress','dir','edge_progress_intervals','vehicle_density','avg_surr_speed','xtrack_dist']]
#save to pickle file
df_list.to_pickle('block_1_sample_features.pkl') # The name should be block4_<day>_<starttime>_feat.pkl eg: block4_24_830_feat.pkl 
df_list.head()

lat        lon    speed  lon_acc  lat_acc        type  \
id traj time                                                                
30 0    0.00  37.976899  23.735836  17.8410  -0.0511  -0.1634  Motorcycle   
        0.04  37.976901  23.735835  17.8382   0.0122  -0.1879  Motorcycle   
        0.08  37.976902  23.735834  17.8428   0.0512  -0.2123  Motorcycle   
        0.12  37.976904  23.735833  17.8530   0.0901  -0.2365  Motorcycle   
        0.16  37.976906  23.735832  17.8714   0.1656  -0.2605  Motorcycle   

              traveled_d  avg_speed   bearing  edge_progress  dir  \
id traj time                                                        
30 0    0.00      439.34  17.080056 -0.086293       0.975860    0   
        0.04      439.34  17.080056 -0.171318       0.974408    0   
        0.08      439.34  17.080056 -0.086292       0.973533    0   
        0.12      439.34  17.080056 -0.086291       0.972081    0   
        0.16      439.34  17.080056 -0.171315       0.970629    0   

              edge_progress_intervals  vehicle_density  avg_surr_speed  \
id traj time                                                             
30 0    0.00                      0.9                5        20.32886   
        0.04                      0.9                5        20.32644   
        0.08                      0.9                5        20.33040   
        0.12                      0.9                5        20.33920   
        0.16                      0.9                5        20.35514   

              xtrack_dist  
id traj time               
30 0    0.00     2.882815  
        0.04     2.830972  
        0.08     2.841368  
        0.12     2.789525  
        0.16     2.737682